Capturando páginas da web
==============

Neste exercício vamos deselvolver algumas técnicas de captura de páginas da web de forma eficiente, alimentando dois bancos de dados com os resultados da nossa captura: O primeiro com metadados da captura e o segundo com o conteúdo das páginas.

Lista de Links
----------------
Baixe a lista de links (https://copy.com/bxNkcbEIsDJ8CWdn) e salve-a no mesmo diretório deste notebook.

**Exercício 1:** Determine o número de links a serem capturados

 


In [2]:
import pandas as pd

**Exercício 2:** Implemente uma solução para visita sequencial às páginas correspodentes aos links.

In [1]:
import requests

**Exercício 3:** Estenda o código do seu capturador para armazenar os seguintes metadados da captura em um banco SQLite: Link, status da requisição, tamanho do html em bytes.

In [4]:
import sqlite3

**Exercício 4:** Implemente o salvamento das páginas capturadas em um banco SQLite, usando o Link como chave primária

**Exercício 5:** Modifique a sua implementação de captura, para que esta seja feita em paralelo. *a)* Usando Threads, *b)* Usando multiplos processos, *c)* Usando a biblioteca gevent.

In [13]:
from threading import Thread
import multiprocessing as MP
import gevent

**Exercício 6:** Faça um profiling das várias metodologias de captura.

In [10]:
import time
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print (
            "@timefn:" + fn.func_name + " took " + str(t2 - t1) + " seconds")
        return result
    return measure_time